In [ ]:
import pickle5 as pickle
with open('../../data/COVID-QA/top_N_ents_spacy-COVID_QA.pkl', 'rb') as f:
    top_N_ents = pickle.load(f)

In [ ]:
import wikipedia
from tqdm.auto import tqdm

search_res = {}

for i, ent in tqdm(enumerate(top_N_ents)):
    #Skipping those entities which don't return anything
    if wikipedia.search(ent) != []:
        search_res[ent] = wikipedia.search(str(ent), results=1)[0]

In [ ]:
import wikipedia
from transformers import AutoTokenizer, AutoModel
import torch

context_dict = {}

filtering = False
if filtering == True:
    filtering_threshold = 0.5
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    #We're consider scibert because pubmedbert assigns very high similarity for both related/unrelated terms.
    checkpoint = 'allenai/scibert_scivocab_uncased'
    
    model = AutoModel.from_pretrained(checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint)
    
    cos = torch.nn.CosineSimilarity(dim=0)
    model.to(device)

for ent,res in tqdm(search_res.items()):
    if filtering == False:
        try:
            context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
        except:
            continue
    else:
        encoded_input = tokenizer([ent, res], return_tensors='pt', padding=True)
        with torch.no_grad():
            output = model(**encoded_input)
        
        similarity = cos(output.pooler_output[0], output.pooler_output[1])
        '''
        we're taking less than here since the similarity scores for related terms seem to be lower than 
        unrelated ones.
        '''
        if similarity.item() < filtering_thresold:
            try:
                context_dict[ent] = wikipedia.page(res, auto_suggest=False).content          
            except:
                continue

In [ ]:
import pandas as pd
pd.DataFrame(context_dict.items(), columns = ['ent', 'text']).to_parquet('wiki_corpus.parquet', index=False)

In [ ]:
from pymed import PubMed

# Create a PubMed object that GraphQL can use to query
# Note that the parameters are not required but kindly requested by PubMed Central
# https://www.ncbi.nlm.nih.gov/pmc/tools/developers/
pubmed = PubMed(tool="MyTool", email="my@email.address")

# Create a GraphQL query in plain text
query = ['covid-19', 'hiv-1']

# Execute the query against the API
results = pubmed.query(query, max_results=500)

# Loop over the retrieved articles
for article in results:

    # Extract and format information from the article
    article_id = article.pubmed_id
    title = article.title
    abstract = article.abstract

    # Show information about the article
    print(
        f'{abstract}\n'
    )

In [2]:
from datasets import load_dataset

In [16]:
s = load_dataset('json', data_files='../../data/policy-qa/train.json')

Using custom data configuration default-e06ad899f49a11fe
Found cached dataset json (/home/saptarshi/.cache/huggingface/datasets/json/default-e06ad899f49a11fe/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [29]:
from pprint import pprint
for article in s['train']["data"][0]:
    title = article.get("title", "")
    for paragraph in article["paragraphs"]:
        for qa in paragraph["qas"]:
            print(pprint(qa))
            
            break
        break
    break

{'answers': [{'answer_start': 90,
              'text': 'personal information you supply when you subscribe, '
                      'order, complete a form or survey, register for one of '
                      'our Web Sites, enter a contest or provide your email '
                      'address and'},
             {'answer_start': 111, 'text': 'you supply'},
             {'answer_start': 90,
              'text': 'personal information you supply when you subscribe, '
                      'order, complete a form or survey, register for one of '
                      'our Web Sites, enter a contest or provide your email '
                      'address'}],
 'id': 'qr0541estkuqixbp',
 'question': 'How do they collect information about users?',
 'type': 'First Party Collection/Use|||Collection Mode|||Explicit'}
None


In [12]:
s = load_dataset('Saptarshi7/PolicyQA')

Using custom data configuration plain_text-3207f4337038390b


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

ValueError: Couldn't cast
version: string
data: list<item: struct<title: string, paragraphs: list<item: struct<qas: list<item: struct<question: string, type: string, id: string, answers: list<item: struct<text: string, answer_start: int64>>>>, index: int64, context: string, summary: list<item: string>>>>>
  child 0, item: struct<title: string, paragraphs: list<item: struct<qas: list<item: struct<question: string, type: string, id: string, answers: list<item: struct<text: string, answer_start: int64>>>>, index: int64, context: string, summary: list<item: string>>>>
      child 0, title: string
      child 1, paragraphs: list<item: struct<qas: list<item: struct<question: string, type: string, id: string, answers: list<item: struct<text: string, answer_start: int64>>>>, index: int64, context: string, summary: list<item: string>>>
          child 0, item: struct<qas: list<item: struct<question: string, type: string, id: string, answers: list<item: struct<text: string, answer_start: int64>>

In [36]:
for article in s['train']['data'][0]:
    print(title)
    for paragraph in article["paragraphs"]:
        context = paragraph["context"]  # do not strip leading blank spaces GH-2585
        print(paragraph['summary'])
        break
    break

sidearmsports.com
['The site collects your generic personal information for a basic service or feature. Collection happens when you explicitly provide information in an unspecified way. You can opt in for data collection for the collection of your information.', 'The site collects your generic personal information for marketing purposes. Collection happens when you explicitly provide information on the website. You can choose not to use the service or feature for the collection of your information.', 'An unnamed third party does collect on the first party website or app unspecified information about you for an unspecified purpose. The identifiability of the data is not covered by our label scheme.', 'The site collects your cookies or tracking elements for an unspecified purpose. Collection happens in an unspecified way, and your data is aggregated or anonymized.', "The site collects your unspecified information for an unspecified purpose. Collection happens when you implicitly provide 

In [19]:
s['train']['data'][1]

IndexError: list index out of range